## Test Probit OWL BART

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.calibration import calibration_curve
# from bart_playground import *
from bart_playground.bart import DefaultBART, ProbitBART, OWLBART, ProbitOWLBART
#, temp #, OWLBART
# from bart_playground.priors import OWLPrior


In [2]:
# Parameters
N_TREES = 50
NDPOST = 1000
NSKIP = 200
RANDOM_STATE = 42

# If debug then run with only one dataset and record running time
# Otherwise run with all datasets
debug = False

In [3]:
def simulate_data(n=30, seed=1):
    np.random.seed(seed)
    X = np.random.uniform(-1, 1, size=(n, 50))
    A = np.random.choice([-1, 1], size=n)
    base = 1 + 2*X[:, 0] + X[:, 1] + 0.5*X[:, 2]
    
    # Scenario 1
    T1 = 0.442 * (1 - X[:, 0] - X[:, 1]) * A
    mu1 = base + T1
    R1 = np.random.normal(loc=mu1, scale=1)

    # Scenario 2
    T2 = (X[:, 1] - 0.25 * (X[:, 0])**2 - 1) * A
    mu2 = base + T2
    R2 = np.random.normal(loc=mu2, scale=1)

    # Scenario 3
    T3 = (0.5 - (X[:, 0])**2 - (X[:, 1])**2) * ((X[:, 0])**2 + (X[:, 1])**2 - 0.3) * A
    mu3 = base + T3
    R3 = np.random.normal(loc=mu3, scale=1)

    # Scenario 4
    T4 = (1 - (X[:, 0])**3 + np.exp((X[:, 2])**2 + X[:, 4]) + 0.6 * X[:, 5] - (X[:, 6] + X[:, 7])**2) * A
    mu4 = base + T4
    R4 = np.random.normal(loc=mu4, scale=1)
    
    # Convert to DataFrames
    colnames = [f"X{i+1}" for i in range(50)]
    base_df = pd.DataFrame(X, columns=colnames)
    base_df["A"] = A

    df1 = base_df.copy()
    df1["R"] = R1

    df2 = base_df.copy()
    df2["R"] = R2

    df3 = base_df.copy()
    df3["R"] = R3

    df4 = base_df.copy()
    df4["R"] = R4

    return {"Scenario 1": df1, "Scenario 2": df2, "Scenario 3": df3, "Scenario 4": df4}

# Print first 5 rows of each scenario
simulated = simulate_data(n=30)
for name, df in simulated.items():
    print(f"\n{name} (first 5 rows):")
    print(df.head())



Scenario 1 (first 5 rows):
         X1        X2        X3        X4        X5        X6        X7  \
0 -0.165956  0.440649 -0.999771 -0.395335 -0.706488 -0.815323 -0.627480   
1 -0.961266  0.357671 -0.576744 -0.468907 -0.016854 -0.893275  0.148235   
2 -0.346710  0.054116  0.771884 -0.285460  0.817070  0.246720 -0.968358   
3 -0.856051  0.934553  0.136201 -0.593414 -0.495349  0.487652 -0.609141   
4  0.900352  0.113306  0.831213  0.283132 -0.219985 -0.028019  0.208621   

         X8        X9       X10  ...       X43       X44       X45       X46  \
0 -0.308879 -0.206465  0.077633  ... -0.439112  0.578559 -0.793548 -0.104213   
1 -0.706543  0.178611  0.399517  ... -0.100176  0.156779 -0.183726 -0.525946   
2  0.858874  0.381794  0.994646  ... -0.880165 -0.757313 -0.910896 -0.785012   
3  0.162718  0.940040  0.693658  ...  0.470132  0.544356  0.815632  0.863944   
4  0.099096  0.852363  0.837467  ...  0.155714 -0.631980  0.575858  0.224062   

        X47       X48       X49       X5

In [4]:
from bart_playground.bart import ProbitOWLBART


df = simulated["Scenario 1"]
X = df[[col for col in df.columns if col.startswith("X")]].values
A = df["A"].values
R = df["R"].values

X_train, X_test, A_train, A_test, R_train, R_test = train_test_split(
    X, A, R, test_size=0.3, random_state=42
)

owlbart = ProbitOWLBART(n_trees=50, ndpost=1000, nskip=200, random_state=42,
                  treatment=A_train, reward=R_train
)
owlbart.fit(X_train, A_train, reward=R_train)
A_opt = owlbart.predict_optimal_treatment(X_test)
R_pred = owlbart.predict_outcome(X_test, A_opt)


Iterations:   0%|          | 0/1200 [00:00<?, ?it/s]

treatment: [ 1 -1 -1 -1  1 -1  1  1 -1  1 -1  1 -1  1 -1 -1 -1 -1 -1  1 -1]
reward: [ 0.59465357  2.29036747  1.58185368 -0.23356383  1.206272   -0.90950869
  2.43393163  0.47279903 -1.04443553  1.16622015 -2.61138451  2.12568281
  0.67725323  1.20886284  0.04709018  1.46566899  2.85783989 -3.24577725
 -1.67131753  1.36845186  1.43814202]


Iterations:   0%|          | 0/1200 [00:02<?, ?it/s]


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'